In [ ]:
import pandas as pd
import numpy as np

def limpieza_data():

    path = "c:/Users/leons/Downloads/prestamos/accepted_2007_to_2018Q4.csv"
    df = pd.read_csv(path)

    # Limpieza de NaNs
    num = 150000
    num_nans = df.isnull().sum()
    columns_menos_nans = num_nans[num_nans < num].index
    df_sin_nans = df[columns_menos_nans]

    df_limpio = df_sin_nans[[
        "id",
        "loan_amnt",
        "funded_amnt",
        "term",
        "int_rate",
        "installment",
        "grade",
        "emp_length",
        "annual_inc",
        "dti",
        "fico_range_high",
        "fico_range_low",
        "revol_util",
        "loan_status",
        "out_prncp",
        "total_rec_prncp",
        "total_rec_int",
        "last_fico_range_high",
        "last_fico_range_low",
        "open_acc",
        "pub_rec",
        "inq_last_6mths",
        "home_ownership",
        "recoveries",
        "num_tl_90g_dpd_24m",
        "purpose",
        "delinq_2yrs"
    ]]

    #Quitamos las dos ultimas filas del dataframe, ya que no nos sirven
    df_limpio = df_limpio.drop(df_limpio.tail(2).index)

    #Limpieza de las filas con nans en base a term, ya que no sirven
    pd.set_option("display.max_rows", 30)
    df_limpio = df_limpio.dropna(subset = ["term"])

    # Columna term a term_months para dejar solo un int8
    df_limpio["term_months"] = pd.to_numeric(df_limpio["term"].str.extract(r"(\d+)")[0], errors = "coerce").astype(np.int8)
    df_limpio = df_limpio.drop(columns=["term"])

    # Columna emp_length a emp_length_years para eliminar la cadena "years"
    df_limpio["emp_length_years"] = df_limpio["emp_length"].str.replace(r"[^0-9+]", "", regex = True)
    df_limpio = df_limpio.drop(columns=["emp_length"])

    return df_limpio

df_limpio = limpieza_data()

In [ ]:
#Optimización de espacio del DataFrame

def optimizar_dataframe(df):
    for columna in df.columns:

        if pd.api.types.is_numeric_dtype(df[columna]):

            if df[columna].dtype == "int64":
                df[columna] = pd.to_numeric(df[columna], downcast = "integer")

            elif df[columna].dtype == "float64":
                df[columna] = pd.to_numeric(df[columna], downcast = "float")  # Puede ser float32 o float16

        elif pd.api.types.is_object_dtype(df[columna]):

            if df[columna].nunique() < 0.5 * len(df[columna]):
                df[columna] = df[columna].astype("category")
    
    return df

df_limpio = optimizar_dataframe(df_limpio)

print(df_limpio.info())

In [5]:
# Pasar DataFrame a CSV
df_limpio.to_pickle("accepted_2007_to_2018Q4_limpio.pkl")

In [ ]:
def segmentar_archivo_binario(input_file, tamano_segmento):
    """
    Segmenta un archivo binario en partes más pequeñas.

    :param input_file: Ruta al archivo binario de entrada
    :param tamano_segmento: Tamaño de cada segmento en bytes
    """
    # Abrimos el archivo binario en modo lectura
    with open(input_file, 'rb') as file:
        i = 0
        while True:
            # Leemos el archivo en bloques del tamaño especificado
            datos = file.read(tamano_segmento)

            # Si no hay más datos, salimos del bucle
            if not datos:
                break

            # Escribimos cada segmento en un nuevo archivo binario
            with open(f'segmento_{i}.pkl', 'wb') as segment_file:
                segment_file.write(datos)

            i += 1
            print(f'Segmento {i} guardado')

segmentar_archivo_binario('accepted_2007_to_2018Q4_limpio.pkl', 48 * 1024 * 1024)  # Segmentos de 48 MB
